# Template for quantitative experiments: text

This template is based on the dianna [text tutorials](https://github.com/dianna-ai/dianna/tree/main/tutorials) for [RISE](https://github.com/dianna-ai/dianna/blob/main/tutorials/rise_text.ipynb) and [LIME](https://github.com/dianna-ai/dianna/blob/main/tutorials/lime_text.ipynb) which are laregely overlapping.

## Loading the Stanford movie reviews model

## Imports and paths

## Specifying the model

## Specifying the model default parameters

## Loading the test data